# PySpark Recipes: Chapter 2: Importing/Reading Data

There are multiple ways to read any file in Spark. We begin with the most popular format CSV and proceed to reading Text and Parquet formats. 

One can read the csv file in Spark using 
- spark.read.csv()
- spark.read.format()

The advisable option is to use the spark.read.format(), and used frequently in this notebook. We have included examples for the other aproaches also in the recipe.

Important parameters for the API csv()-<br/> (adapted from - https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html)

<i>csv(path, schema=None, sep=None, encoding=None, quote=None, escape=None, comment=None, header=None, 
   inferSchema=None, ignoreLeadingWhiteSpace=None, ignoreTrailingWhiteSpace=None, nullValue=None, 
   nanValue=None, positiveInf=None, negativeInf=None, dateFormat=None, timestampFormat=None, maxColumns=None, 
   maxCharsPerColumn=None, maxMalformedLogPerPartition=None, mode=None, columnNameOfCorruptRecord=None, 
   multiLine=None, charToEscapeQuoteEscaping=None, samplingRatio=None, enforceSchema=None, emptyValue=None)</i>

- path – string, or list of strings, for input path(s).
- schema – an optional pyspark.sql.types.StructType for the input schema.
- header – uses the first line as names of columns. If None is set, it uses the default value, false.
- inferSchema – infers the input schema automatically from data. It requires one extra pass over the data. If None is set, it uses the default value, false.
- sep (if using spark.read.csv() – sets the single character as a separator for each field and value. If None is set, it uses the default value, ,. (note: one can also use "delimiter")
- encoding – decodes the CSV files by the given encoding type. If None is set, it uses the default value, UTF-8.
- dateFormat – sets the string that indicates a date format. Custom date formats follow the formats at java.text.SimpleDateFormat. This applies to date type. If None is set, it uses the default value, yyyy-MM-dd.
- timestampFormat – sets the string that indicates a timestamp format. Custom date formats follow the formats at java.text.SimpleDateFormat. This applies to timestamp type. If None is set, it uses the default value, yyyy-MM-dd'T'HH:mm:ss.SSSXXX.
- quote – sets the single character used for escaping quoted values where the separator can be part of the value. If None is set, it uses the default value, ". If you would like to turn off quotations, you need to set an empty string.
- escape – sets the single character used for escaping quotes inside an already quoted value. If None is set, it uses the default value, \.
- ignoreLeadingWhiteSpace – A flag indicating whether or not leading whitespaces from values being read should be skipped. If None is set, it uses the default value, false.
- ignoreTrailingWhiteSpace – A flag indicating whether or not trailing whitespaces from values being read should be skipped. If None is set, it uses the default value, false.
- nullValue – sets the string representation of a null value. If None is set, it uses the default value, empty string. Since 2.0.1, this nullValue param applies to all supported types including the string type.
- nanValue – sets the string representation of a non-number value. If None is set, it uses the default value, NaN.

- comment – sets the single character used for skipping lines beginning with this character (e.g. "#"). By default (None), it is disabled. 
- maxColumns – defines a hard limit of how many columns a record can have. If None is set, it uses the default value, 20480.
- maxCharsPerColumn – defines the maximum number of characters allowed for any given value being read. If None is set, it uses the default value, -1 meaning unlimited length.
- mode - this one is interesting, by default the mode is "PERMISSIVE", sets other fields to null when it meets a corrupted record. When a length of parsed CSV tokens is shorter than an expected length of a schema, it sets null for extra fields. "DROPMALFORMED" would ignore the whole corrupted records, and "FAILFAST" would throw an exception, the moment you meet the corrupted records. 
- multiLine – parse records, which may span multiple lines. If None is set, it uses the default value, false.

nullValue and nanValue would follow default behaviour - empty string would be taken as string, and non-number value would be treated as NaN.

Enough theory - now let us get back to practicals :).

In [ ]:
import pyspark
import time
from pyspark.sql import SparkSession


# creating a SparkSession object - you can change any of the configuration option you like. Remember this would
# get the existsing SparkSession and would not create a new one.
# So in case your previous notebook is still running - no issues
sparkSession = SparkSession \
                .builder \
                .master("local") \
                .appName("Pyspark Recipes - Importing Data") \
                .getOrCreate()

startTime = time.time()

# Remember in the previous session we had imported a CSV file
dfCensus = sparkSession.read.format('csv') \
            .options(header = True, inferSchema = True, sep = ",", enforceSchema = True,
                ignoreLeadingWhiteSpace = True, ignoreTrailingWhiteSpace = True) \
            .load('../datasets/charityml/censusdata.csv')

print(dfCensus.count())
print(time.time()-startTime)



Now to get the Schema of Datatypes (dtypes) of the dataframe below are the two methods.


In [ ]:
#This would give us the Schema of the text file
dfCensus.printSchema()

#One can also get just the datatypes by using - 
dfCensus.dtypes


<span style="color:green;"><i>Extra Gyaan</i></span>

At least till now, there is not an elegant way to know a datatype of a single column. If there is a situtation 
where you have to check the datatype of a single column, this is the trick we adopt.


In [ ]:
dict(dfCensus.dtypes)['education-num']

## OR ##
# uncomment the following 3 lines

#def getDataType(df,columnName):
#    return [dtype for name, dtype in df.dtypes if name == columnName][0]

#getDataType(dfCensus,'income')



<span style="color:green;"><i>Extra Gyaan</i></span>

This would be typical way to read a CSV file. However, while reading a text file, without defining a schema would require two pass through the entire file, so that the schema can be applied across. Spark unfortunately does not infer the schema using a sample. This could lead to time taking more time than usual than reading a large say 10 GB file. 

We have a nifty small code which help us infer the schema, which we are sharing with you. The steps you need ot take are 
- You can take a sample 10,100,1000 rows, instead of taking the entire large file,
- Check the schema, make changes if necessary
- Reimport the file with the defined schema

** Remember this is complete optional, but could save time **


In [ ]:
# here we get a list of 10 rows from the CSV file

startTime = time.time()
lstSample = sparkSession.read.format('csv') \
            .options(header = True, inferSchema = True, delimiter = ",") \
            .load('../datasets/charityml/censusdata.csv').head(10)

print(time.time()-startTime)

lstSample

In [ ]:
import pyspark.sql.types as pst#pst stands for (p)yspark(s)ql(t)ypes
from pyspark.sql import Row


# a function to help infer schema
def infer_schema(rec):
    """infers dataframe schema for a record. Assumes every dict is a Struct, not a Map"""
    if isinstance(rec, dict):
        return pst.StructType([pst.StructField(key, infer_schema(value), True)
                              for key, value in sorted(rec.items())])
    elif isinstance(rec, list):
        if len(rec) == 0:
            raise ValueError("can't infer type of an empty list")
        elem_type = infer_schema(rec[0])
        for elem in rec:
            this_type = infer_schema(elem)
            if elem_type != this_type:
                raise ValueError("can't infer type of a list with inconsistent elem types")
        #return pst.ArrayType(elem_type)
        return elem_type
    else:
        return pst._infer_type(rec)



In [ ]:
schema = infer_schema(lstSample)
schema

In [ ]:
startTime = time.time()

# Remember in the previous session we had imported a CSV file
dfCensus = sparkSession.read.format('csv') \
            .options(header = True, inferSchema = True, sep = ",", enforceSchema = True,
                ignoreLeadingWhiteSpace = True, ignoreTrailingWhiteSpace = True) \
            .schema(schema) \
            .load('../datasets/charityml/censusdata.csv')

print(dfCensus.count())
print(time.time()-startTime)

Check the import time while reading a text file, with schema defined, versus schema not defined. Feel free to use any of the important options that are available with the .csv() method

Another tip to improve your operation is to use parquet file. Any intermediate files that needs to be written back to disk, should ideally be written in parquet format. There are two main benefits -
- The file stored is very compressed, for example, the near about 5-6 MB file we are reading would take around 1-1.5 MB space, if written back in parquet format
- The file is stored with schema, one does not have to worry about infering schema



In [ ]:
dfCensus.write.parquet('census.parquet', mode='overwrite')

startTime = time.time()
dfCensus = sparkSession.read.parquet('census.parquet')
print(time.time()-startTime)

In [ ]:
dfCensus.show(n=5)


We would continue to work on this dataframe, in our next notebook we would look at EDAs (Exploratory Data Analysis)
